# Cox rings of blow-ups of points

Author: [Mara Belotti](https://page.math.tu-berlin.de/~belotti/)

updated to v1.0 by Michael Joswig

Based on: Belotti, Panizzut. [Discrete geometry of Cox rings of blow-ups of $\mathbb{P}^3$, arXiv:2208.05258](https://arxiv.org/abs/2208.05258).

This notebook explains the necessary OSCAR <font color=blue>[Oscar]</font> code used to prove the result in the aforementioned paper through a *simplified example* with the same flavor. The version of the code needed in the paper can be found in this [github repository](https://github.com/marabelotti/7PointsPP3).

In [1]:
using Oscar

  ___   ____   ____    _    ____
 / _ \ / ___| / ___|  / \  |  _ \   |  Combining ANTIC, GAP, Polymake, Singular
| | | |\___ \| |     / _ \ | |_) |  |  Type "?Oscar" for more information
| |_| | ___) | |___ / ___ \|  _ <   |  Manual: https://docs.oscar-system.org
 \___/ |____/ \____/_/   \_\_| \_\  |  Version 1.0.0


## Some background

We consider a $\mathbb{C}(t)$-subalgebra $R$ of a polynomial ring $\mathbb{C}(t)[z_1,\dots,z_m]$. The *order* $\text{ord}(f)$ of an element $f$ in $\mathbb{C}(t)[z_1,\dots,z_m]$ is the smallest integer ord$(f)$  such that $t^{-\text{ord}(f)}f$ has neither a pole or a zero. The initial form of $f$ is given by the polynomial 
$$\text{in}(f)=(t^{-\text{ord}(f)}f)|_{t=0} \in \mathbb{C}[z_1,\dots,z_m].$$
For a collection $S$ of polynomials, we indicate with $\text{in}(S)$ the set of initial forms of the elements in $S$. 

**Definition** (*Khovanskii Basis*) A *Khovanskii basis* for $R$ is a set of generators $\mathcal{F}\subset R$ such that $\text{in}(f)$ is a monomial for every $f\in\mathcal{F}$ and such that the $\mathbb{C}$-algebra $\text{in}(R)$ is generated by the initial forms in $\text{in}(\mathcal{F})$.

Let us see, for example, how to compute these initial forms using OSCAR. The function ``leading_t`` takes as an input a polynomial ``eq`` in some multivariate polynomial ring of type ``FmpqMPolyRing``. This polynomial ring must have a variable named ``t``. The output is the initial form of ``eq``.

In [3]:
function leading_t(eq::QQMPolyRingElem)

    if(length(eq)==0)
        return eq,true;
    end
    
    sort_terms!(eq)
    deg_lead=degree(term(eq,length(eq)),t)
    lead=coeff(eq,[1],[deg_lead])
    monomial=is_monomial(lead)
    return lead;
end

leading_t (generic function with 1 method)

Cox rings $\text{Cox}(\text{Bl}_k(\mathbb{P}^d))$ of blow-ups of $k$ points in $\mathbb{P}^d$ <sup><font color=green>[1]</font></sup> are subalgebras of polynomial rings $\mathbb{C}[x_1,\dots,x_k,y_1,\dots,y_k]$. In every case $(k,d)\neq (8,4)$ and for every choice of $k$ points the generators of this subalgebras are known explicitely . However, in the case $(7,3)$ the complete ideal of the relations between these generators was not studied. In <font color=blue>[BelPan]</font> we showed that this ideal is generated by quadrics for every choice of points *in general position*. 
This could in principle be achieved by a Gröbner basis computation <font color=blue>[Alg]</font>. However, the worst case complexity of such a computation is doubly exponential, and it is not feasible in our situation.
We therefore search for a sufficiently general choice of points whose correspoding Cox ring generators are a  Khovanskii Basis for the subalgebra.

In this notebook, we will show how to reproduce the same computations for the case of $6$ points in $\mathbb{P}^3$, for which the result of quadratic generation of the ideal was already known from <font color=blue>[StuXu]</font>. We will work with $\mathbb{C}(t)$-algebras, which allows us to prove the result in $\mathbb{C}$, as explained in <font color=blue>[BelPan]</font>. 

<sup><font color=green>[1]</font></sup> <font size="1"> *the precise definition is not needed for the goal of this notebook* </font>

## Cox ring of $6$ points in $\mathbb{P}^3$

The Cox ring of the blow-up of $6$ points in $\mathbb{P}^3$ is a multigraded subalgebra of $\mathbb{C}(t)[x_1,x_2,x_3,x_4,x_5,x_6,y_1,y_2,y_3,y_4,y_5,y_6]$. A choice of $6$ points in $\mathbb{P}^3$ is represented by a $4\times 6$ matrix with entries in $\mathbb{C}(t)$. 

The multigrade is derived from the following $\mathbb{Z}^7$-multigrading of the polynomial ring:
* $\text{deg}(x_i)=e_{i}$
* $\text{deg}(y_i)=e_0+e_i$
where $e_{0},\dots,e_{6}$ is the canonical basis for the lattice $\mathbb{Z}^7$.

The generators of the Cox ring $\text{Cox}(\text{Bl}_6(\mathbb{P}^3))$ are the following:
* <font color=green>(*exceptional sections*) 6 elements</font>: x1, x2, x3, x4, x5, x6
* <font color=orange> (*hyperplane sections*) 20 elements </font> constructed in the following way. For every triple of distinct points $p_{i_1},p_{i_2},p_{i_3}$ consider the equation $H(z_1,z_2,z_3,z_4)$ of the hyperplane passing through those three points. Then the generator is given by
$$H(P\cdot\frac{y}{x})\frac{x_1x_2x_3x_4x_5x_6}{x_{i_1}x_{i_2}x_{i_3}}$$
where $\cdot$ is the standard matrix-scalar product and $\frac{y}{x}$ is the vector whose entries are given by $\frac{y_i}{x_i}$.
* <font color=violet> (*quadric sections*) 6 elements </font> constructed in the following way. For every point $p_{i_1}$ consider the equation $Q(z_1,z_2,z_3,z_4)$ of the quadric passing through all points and with multiplicity two at $p_{i_1}$. Then the generator is given by
$$Q(P\cdot\frac{y}{x})\frac{x_1x_2x_3x_4x_5x_6}{x_{i_1}}$$
where $\cdot$ is the standard matrix-scalar product and $\frac{y}{x}$ is the vector whose entries are given by $\frac{y_i}{x_i}$.

This is the polynomial ring where our subalgebra lives


In [4]:
R, (t,x1,x2,x3,x4,x5,x6,y1,y2,y3,y4,y5,y6) = polynomial_ring(QQ, ["t",["x[$i]" for i in 1:6]...,["y[$i]" for i in 1:6]...])

(Multivariate polynomial ring in 13 variables over QQ, QQMPolyRingElem[t, x[1], x[2], x[3], x[4], x[5], x[6], y[1], y[2], y[3], y[4], y[5], y[6]])

We define a matrix ``P`` which encodes our choice of $6$ points

In [5]:
P=[t^59 t^44 t^79 t^20 t^12 t^81; t^8 t^72 t^49 t^39 t^58 t^23; t^44 t^58 t^12 t^52 t^57 t^49; t^25 t^23 t^60 t^72 t^45 t^51]

4×6 Matrix{QQMPolyRingElem}:
 t^59  t^44  t^79  t^20  t^12  t^81
 t^8   t^72  t^49  t^39  t^58  t^23
 t^44  t^58  t^12  t^52  t^57  t^49
 t^25  t^23  t^60  t^72  t^45  t^51

The generators deeply depend on the choice of representatives of the projective points. Therefore, dividing by the lowest degree element in every column does not guarantee the same result and, in this case, we would not get a Khovanskii basis.

## Some helper functions

We create a function ``sub_linear`` which given as an input a polynomial ``eq`` in $x_1,\dots,x_6$ and the matrix of the points ``P`` returns an element in the fraction ring of $\mathbb{C}(t)[x_1,x_2,x_3,x_4,x_5,x_6,y_1,y_2,y_3,y_4,y_5,y_6]$ obtained by applying the map
$$ (x_1,x_2,x_3,x_4,x_5,x_6)\to P\cdot \frac{y}{x}$$
This is just a technical function needed to compute the generators

In [16]:
function sub_linear(eq::QQMPolyRingElem,P::Matrix{QQMPolyRingElem})
    linear_forms=Vector{AbstractAlgebra.Generic.FracFieldElem{QQMPolyRingElem}}(P*[gens(eq.parent)[i+6]//gens(eq.parent)[i] for i in range(2,7)])
    sub = hom(eq.parent, fraction_field(eq.parent), z -> z, [eq.parent(t),linear_forms..., gens(eq.parent)[size(P)[1]+2:end]...])
    return sub(eq)
end

sub_linear (generic function with 1 method)

The function ``veronese`` is another technical function needed to compute the quadrics sections generators. This implements the following map:
$$[v_1,\dots, v_n]\to [v_iv_j\text{ for }0\leq i\leq j\leq n] $$

In [7]:
function veronese(v::Vector{QQMPolyRingElem})
    return [v[i]*v[j] for i in 1:length(v) for j in i:length(v)]
end

veronese (generic function with 1 method)

The function ``leading_t_vect`` is a version of ``leading_t`` for vectors of polynomials

In [8]:
function leading_t_vect(eq_vect::Vector{QQMPolyRingElem})
    return  (leading_t.(eq_vect)), all((length.(leading_t.(eq_vect))).<2);
end

leading_t_vect (generic function with 1 method)

## Creation of generators

In this section we create the generators for $\text{Cox}(\text{Bl}_6\mathbb{P}^3)$ described above

In [9]:
function exceptional_sections(P::Matrix{QQMPolyRingElem})
    exc=Vector{QQMPolyRingElem}([gens(R)[i] for i in 2:7])
    deg_mon_exc=[[j==i+1 ? 1 : 0 for j in 1:7] for i in 1:6]
    return exc,deg_mon_exc
end

exceptional_sections (generic function with 1 method)

In [10]:
function hyperplanes_sections(P::Matrix{QQMPolyRingElem})

    @assert size(P)==(4,6)
    
    Hyp=Vector{QQMPolyRingElem}([det(matrix(R,[[gens(R)[2:5]]; [R.(P)[:,s] for s in (i,j,k)]])) for i in 1:6 for j in i+1:6 for k in j+1:6]);
    
    Hyp=map(y->numerator(sub_linear(y,P)),Hyp)
    deg_mon_hyp=[[((v==i+1) | (v==j+1) | (v==k+1)) ? 0 : 1 for v in 1:7] for i in 1:6 for j in i+1:6 for k in j+1:6];
    
    return Hyp , deg_mon_hyp
end

hyperplanes_sections (generic function with 1 method)

In [11]:
function quadrics_sections(P::Matrix{QQMPolyRingElem})

    @assert size(P)==(4,6)

    Quad=Vector{QQMPolyRingElem}()
    
    for j in 1:6
        #j is the index of the double point
        temp=[veronese(gens(R)[2:5])]

        for ind_row in 1:6
            F = hom(R, R, z -> z, [t,P[:,ind_row]...,gens(R)[6:end]...])
            if (ind_row!=j) 
                push!(temp,F.(veronese(gens(R)[2:5])))
            elseif (ind_row==j)
                for var in gens(R)[2:5]
                    push!(temp,F.(derivative.(veronese(gens(R)[2:5]),var)))
                end
            end
        end
        push!(Quad,det(matrix(R,temp)))
    end

     
    Quad=map(y->numerator(sub_linear(y,P)),Quad)

    deg_mon_quad=[[((v==1) | (v==i+1)) ? 2 : (v==j+1 ? 0 : 1) for v in 1:7] for i in 1:6 for j in ((1:i-1)...,(i+1:6)...)]
    
    return Quad , deg_mon_quad
end

quadrics_sections (generic function with 1 method)

## Verifying generators form a Khovanskii basis

The function ``generalposition_monomeric_check`` produces the generators described above. The output of the function is:

* A boolean ``bool`` which is true if and only if the seven points given by the columns of ``P`` are in *general position*, meaning none of the generators is the identically zero polynomial.


* If bool is true, a vector of integers ``deg_mon`` which gives the multidegree of the 32 generators.

* If bool is true, a vector of monomials in ``R`` which are the initial forms of the generators.


In [12]:
function generalposition_monomeric_check(P::Matrix{QQMPolyRingElem})
    
    exc, deg_mon_exc=exceptional_sections(P);
    exc_mon,bol_exc=leading_t_vect(exc);

    hyps, deg_mon_hyp= hyperplanes_sections(P);
    hyps_mon,bol_hyps= leading_t_vect(hyps);
    
    quad,deg_mon_quad = quadrics_sections(P);
    quad_mon,bol_quad= leading_t_vect(quad);
    
    if !all(hyps .!=R(0)) | !all(quad .!=R(0)) 
        println("One of the generators is zero, therefore the points are not in general position")
        return false;
    end
    if !bol_hyps | !bol_quad
        println("One of the initial forms is not monomeric, therefore the generators are not a Khovanskii basis")
        return false;
    end
    
    deg_mon=[deg_mon_exc...,deg_mon_hyp...,deg_mon_quad...]
    equations_mon=[exc_mon...,hyps_mon...,quad_mon...];

    return true,deg_mon,equations_mon;
end


generalposition_monomeric_check (generic function with 1 method)

Let us apply it to the matrix ``P`` defined above 

In [17]:
bool,deg_mon,initial_forms= generalposition_monomeric_check(P)

(true, [[0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 1, 1, 1], [1, 0, 0, 1, 0, 1, 1], [1, 0, 0, 1, 1, 0, 1], [1, 0, 0, 1, 1, 1, 0]  …  [2, 0, 1, 1, 1, 2, 1], [2, 1, 0, 1, 1, 2, 1], [2, 1, 1, 0, 1, 2, 1], [2, 1, 1, 1, 0, 2, 1], [2, 1, 1, 1, 1, 2, 0], [2, 0, 1, 1, 1, 1, 2], [2, 1, 0, 1, 1, 1, 2], [2, 1, 1, 0, 1, 1, 2], [2, 1, 1, 1, 0, 1, 2], [2, 1, 1, 1, 1, 0, 2]], QQMPolyRingElem[x[1], x[2], x[3], x[4], x[5], x[6], -x[4]*x[6]*y[5], x[5]*x[6]*y[3], x[4]*x[6]*y[3], -x[3]*x[4]*y[5]  …  x[2]*x[6]*y[1], -x[1]*x[5]*y[2], -x[1]*x[4]*y[2], -x[1]*x[2]*y[3], 2*x[2]*x[4]*x[6]*y[3]*y[5], 2*x[1]*x[4]*x[6]*y[3]*y[5], -2*x[2]*x[4]*x[6]*y[1]*y[5], 2*x[2]*x[5]*x[6]*y[1]*y[3], 2*x[2]*x[4]*x[6]*y[1]*y[3], 2*x[1]*x[2]*x[4]*y[3]*y[5]])

This tells us that our points are in general positions and the initial forms are monomeric. We now verify that the Hilbert function of the Cox ring and of the initial algebra coincide in degree $2$.
The function ``degree_two_check`` takes as an input ``deg_mon`` and ``initial_forms`` from the function ``generalposition_monomeric_check`` and returns true if the Hilbert functions coincide in degree two or false if not.

In [18]:
function degree_two_check(deg_mon::Vector{Vector{Int64}},initial_forms::Vector{QQMPolyRingElem})
    
    couples=[[initial_forms[i],initial_forms[j],i,j] for i in 1:32 for j in i:32]
    deg_couples=[deg_mon[i].+deg_mon[j] for i in 1:32 for j in i:32]

    dimensions=[[2 for i in 1:15]...,4]
    orbit_degreetwo=[[[ k==i || k==j ? 0 : 1 for k in 1:7] for i in 2:7 for j in i+1:7]...,
        [2, 1, 1, 1, 1, 1, 1, 1]]

    bol_hilb=true
    for (deg,cont_deg) in zip(orbit_degreetwo,range(1,2))
        monomials=[]

        for (cou,i) in zip(couples,range(1,length(couples)))
            if deg_couples[i]==deg
                push!(monomials,cou[2]*cou[1]);
            end
        end

        if length(monomials)!=1
            monomials_mon=Set()
            for el in monomials
                push!(monomials_mon,monomial(el,1))
            end
            if length(monomials_mon)!=dimensions[cont_deg]
                println(deg)
                println(monomials_mon)
                bol_hilb=false
            end
        end
    end
    
    if !bol_hilb
        println("Not Khovanskii cause Hilbert function of initial algebra doesn't match")
        return false;
    end
        
    return true
end

degree_two_check (generic function with 1 method)

In [19]:
degree_two_check(deg_mon,initial_forms)

true

The last thing left to verify is that the relations between the initial forms are generated by quadrics. This guarantees that the Hilbert functions coincide not only for degree two, but in every degree. To do this, we use the function ``matrix_of_exponents`` which creates a $32\times 12$ matrix where every row corresponds to an initial form and the column register the exponents for the $12$ variables.

In [20]:
function matrix_of_exponents(initial_forms::Vector{QQMPolyRingElem})
    Ring=initial_forms[1].parent;
    M=zero_matrix(ZZ,32,12);
    for (eq,row) in zip(initial_forms,range(1,32))
        for (var,col) in zip(gens(Ring)[2:end],range(1,12))
            M[row,col]=Oscar.degree(eq,var)
        end
    end
    return M; 
end

matrix_of_exponents (generic function with 1 method)

In [23]:
S = matrix_space(ZZ,32,12)
M = S(matrix_of_exponents(initial_forms))

[1   0   0   0   0   0   0   0   0   0   0   0]
[0   1   0   0   0   0   0   0   0   0   0   0]
[0   0   1   0   0   0   0   0   0   0   0   0]
[0   0   0   1   0   0   0   0   0   0   0   0]
[0   0   0   0   1   0   0   0   0   0   0   0]
[0   0   0   0   0   1   0   0   0   0   0   0]
[0   0   0   1   0   1   0   0   0   0   1   0]
[0   0   0   0   1   1   0   0   1   0   0   0]
[0   0   0   1   0   1   0   0   1   0   0   0]
[0   0   1   1   0   0   0   0   0   0   1   0]
[0   0   0   0   1   1   0   1   0   0   0   0]
[0   0   0   1   0   1   0   1   0   0   0   0]
[0   1   0   1   0   0   0   0   0   0   1   0]
[0   1   0   0   0   1   0   0   1   0   0   0]
[0   1   0   0   1   0   0   0   1   0   0   0]
[0   1   0   1   0   0   0   0   1   0   0   0]
[0   0   0   0   1   1   1   0   0   0   0   0]
[0   0   0   1   0   1   1   0   0   0   0   0]
[1   0   0   1   0   0   0   0   0   0   1   0]
[1   0   0   0   0   1   0   0   1   0   0   0]
[1   0   0   0   1   0   0   0   1   0  

Using the command ``markov4ti2`` imported in Oscar from the software 4ti2 <font color=blue>[4ti2]</font> we compute generators for the toric ideal that gives the relations between the initial forms

In [28]:
S, (e1,e2,e3,e4,e5,e7,H123,H124,H125,H126,H134,H135,H136,H145,H146,H156,H234,H235,H236,H245,H246,H256,H345, H346,H356,H456,Q1,Q2,Q3,Q4,Q5,Q6) = polynomial_ring(QQ, ["e1","e2","e3","e4","e5","e6","H123","H124","H125","H126","H134","H135","H136","H145","H146","H156","H234","H235","H236","H245","H246","H256","H345","H346","H356","H456","Q1","Q2","Q3","Q4","Q5","Q6"])

(Multivariate polynomial ring in 32 variables over QQ, QQMPolyRingElem[e1, e2, e3, e4, e5, e6, H123, H124, H125, H126  …  H345, H346, H356, H456, Q1, Q2, Q3, Q4, Q5, Q6])

In [29]:
toric_ideal_of_initial_forms = binomial_exponents_to_ideal(S, Oscar.markov4ti2(transpose(kernel(transpose(M))[2])))

LoadError: ArgumentError: linear indexing not supported for non-vector matrices

This last loop verify that all the generators of this toric ideal have degree $2$.

In [18]:
for el in gens(toric_ideal_of_initial_forms)
    if total_degree_fmpz(el)!=2
            println("generator ", el, " has degree ", total_degree_fmpz(el))
    end
end

## Bibliography

<font color=blue>[StuXu]</font> Bernd Sturmfels, Zhiqiang Xu, Sagbi bases of Cox–Nagata rings. J. Eur. Math. Soc. 12 (2010), no. 2, pp. 429–459

<font color=blue>[BelPan]</font> Mara Belotti, Marta Panizzut, [Discrete geometry of Cox rings of blow-ups of $\mathbb{P}^3$, arXiv:2208.05258](https://arxiv.org/abs/2208.05258).

<font color=blue>[Oscar]</font>
    OSCAR -- Open Source Computer Algebra Research system, Version 1.0 The OSCAR Team, 2024. (https://www.oscar-system.org)
    
<font color=blue>[4ti2]</font>
    4ti2---A software package for algebraic, geometric and
            combinatorial problems on linear spaces, 4ti2 team. (https://4ti2.github.io)

<font color=blue>[Alg]</font>
    David A. Cox, John Little, Donal O’Shea, Ideals, Varieties, and Algorithms. Springer New York, 2007.